#### Instalacion Librerias 

In [4]:
pip install google-cloud-storage

Note: you may need to restart the kernel to use updated packages.


#### Librerias 

In [5]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import win32com.client as win32
import pandas as pd

from google.cloud import storage



#### Proceso RPA

In [6]:


# Configuración del navegador
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized") 
options.add_argument("--disable-blink-features=AutomationControlled")  

# Inicia el navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Carga la URL
driver.get("https://sicoe.com.co/sicoe/dist/#/")

# Espera a que cargue el botón "Iniciar sesión"
boton_login = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((
        By.XPATH,
        '//a[contains(text(), "Iniciar sesión")]'
    ))
)

boton_login.click()

#Acceso de informacion a la pagina
input_nit = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "nit"))
    )

# Escribir el NIT en el campo
input_nit.send_keys("8301256101")

# Espera a que cargue el campo de usuario
input_usuario = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.ID, "login"))
    )

input_usuario.send_keys("analistadatos")

# Espera a que cargue el campo de contraseña
input_pass = WebDriverWait(driver, 15).until(
EC.presence_of_element_located((By.ID, "passwd"))
    )
input_pass.send_keys("628473*****Se")

#Manejo de Checkbok 
checkbox = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="checkbox"]'))
    )
checkbox.click()

# Darle Click al boton iniciar sesion

boton_iniciar = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Iniciar")]'))
    )
boton_iniciar.click()

#Click en el Boton de Informe 
boton_informes = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[href="/sam/menu1/menu/index.php"]'))
    )
boton_informes.click()


#Click en el boton siguiente 
boton_next = WebDriverWait(driver, 15).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.buttonNext'))
    )
boton_next.click()


#Click en el boton de rutas 
rutas_por_vendedor = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//a[@class='tip' and contains(@href, \"ruta_x_asesor\")]"))
    )
rutas_por_vendedor.click()

#Click en exportar a excel 
WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.ID, "sb-player"))
    )
print("✅ Cambiado al iframe con el informe.")

boton_excel = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "excel"))
    )
boton_excel.click()

time.sleep(20)  # Espera para asegurarse de que la descarga se complete
# Cierra el navegador y finaliza el WebDriver
driver.quit()
print("✅ Navegador cerrado.")

NameError: name 'Service' is not defined

#### Descarga - Transformacion de Archivos 

In [ ]:


directorio = r"C:\Users\dani\Downloads"

def buscar_archivo_rutas(directorio):
    archivos = [
        f for f in os.listdir(directorio)
        if "rutas_x_asesor" in f and f.endswith(".xls")
    ]
    if not archivos:
        print("❌ No se encontró ningún archivo que coincida.")
        return None

    archivos.sort(key=lambda f: os.path.getmtime(os.path.join(directorio, f)))
    archivo = archivos[-1]
    ruta_completa = os.path.join(directorio, archivo)
    print("✅ Archivo encontrado:", ruta_completa)
    return ruta_completa

# Buscar archivo
ruta_archivo = buscar_archivo_rutas(directorio)

# Leer con pandas usando engine correcto



def convertir_a_xlsx(ruta_xls):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(ruta_xls)
    nueva_ruta = ruta_xls + "x"  # Agrega una "x" para .xlsx
    wb.SaveAs(nueva_ruta, FileFormat=51)  # 51 = formato .xlsx
    wb.Close()
    excel.Quit()
    print("✅ Archivo convertido a:", nueva_ruta)
    return nueva_ruta

# Convertir y leer con pandas
ruta_xlsx = convertir_a_xlsx(ruta_archivo)
df = pd.read_excel(ruta_xlsx)


✅ Archivo encontrado: C:\Users\Analista de  Datos\Downloads\rutas_x_asesor20250801.xls
✅ Archivo convertido a: C:\Users\Analista de  Datos\Downloads\rutas_x_asesor20250801.xlsx


In [ ]:

# === CONFIGURACIÓN ===
ruta_credenciales = r'credentials/croc-454221-e1a3c2e02181.json'
bucket_name = 'bucket-quickstart_croc_830'
carpeta_destino = 'raw/Ventas/sicoe_rutas_vendedor'
directorio = r"C:\Users\Analista de  Datos\Downloads"

# === FUNCIÓN PARA BUSCAR ARCHIVO MÁS RECIENTE ===
def buscar_archivo_rutas(directorio):
    archivos = [
        f for f in os.listdir(directorio)
        if "rutas_x_asesor" in f and f.endswith(".xlsx")
    ]
    if not archivos:
        print("❌ No se encontró ningún archivo que coincida.")
        return None
    archivos.sort(key=lambda f: os.path.getmtime(os.path.join(directorio, f)))
    archivo = archivos[-1]
    return os.path.join(directorio, archivo)

# === ENCONTRAR ARCHIVO
archivo_local = buscar_archivo_rutas(directorio)
if not archivo_local:
    raise FileNotFoundError("No se encontró el archivo .xlsx")

# === AUTENTICACIÓN
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ruta_credenciales
cliente = storage.Client()
bucket = cliente.bucket(bucket_name)

# === ELIMINAR ARCHIVOS EXISTENTES EN LA CARPETA
print(f"🧹 Limpiando carpeta '{carpeta_destino}' en el bucket...")
blobs = bucket.list_blobs(prefix=carpeta_destino)
for blob in blobs:
    print(f"🗑 Eliminando: {blob.name}")
    blob.delete()

# === SUBIR NUEVO ARCHIVO
nombre_archivo_bucket = os.path.join(carpeta_destino, os.path.basename(archivo_local)).replace("\\", "/")
blob = bucket.blob(nombre_archivo_bucket)
blob.upload_from_filename(archivo_local)

print(f"✅ Archivo subido exitosamente: {nombre_archivo_bucket}")



🧹 Limpiando carpeta 'raw/Ventas/sicoe_rutas_vendedor' en el bucket...
🗑 Eliminando: raw/Ventas/sicoe_rutas_vendedor/
🗑 Eliminando: raw/Ventas/sicoe_rutas_vendedor/rutas_x_asesor20250801.xlsx
✅ Archivo subido exitosamente: raw/Ventas/sicoe_rutas_vendedor/rutas_x_asesor20250801.xlsx


In [ ]:
import os

# Ruta a tu carpeta de Descargas
directorio = r"C:\Users\Analista de  Datos\Downloads"

# Buscar y eliminar archivos que contengan 'rutas_x_asesor'
archivos_eliminados = 0
for archivo in os.listdir(directorio):
    if "rutas_x_asesor" in archivo:
        ruta_completa = os.path.join(directorio, archivo)
        try:
            os.remove(ruta_completa)
            print(f"🗑 Eliminado: {ruta_completa}")
            archivos_eliminados += 1
        except Exception as e:
            print(f"⚠️ Error al eliminar {archivo}: {e}")

if archivos_eliminados == 0:
    print("📁 No se encontraron archivos para eliminar.")
else:
    print(f"✅ Eliminados {archivos_eliminados} archivos.")


🗑 Eliminado: C:\Users\Analista de  Datos\Downloads\rutas_x_asesor20250801.xls
🗑 Eliminado: C:\Users\Analista de  Datos\Downloads\rutas_x_asesor20250801.xlsx
✅ Eliminados 2 archivos.
